# Finding Matrices for Rational Householder Decompositions

QR decomposition is a widely discussed topic in the realms of Linear Algebra. If you've ever taught someone how to factorize a 4x4 matrix and came across some algebraic obstacles when teaching the Gram-Schmidt or Householder method, finding yourself lost in a a series of square-root calculations (which had to be done wothout the aid of a calculator), we have an algorithm that could make your life a little easier. 

The program was written in Python with the aid of the symbolic computing library sympy, and what we do is search a finite list for values that would make "good" columns for our input matrix. Then, we test one of the possible so called rational matrices against the Householder factorizing algorithm and check if it in fact renders rational calculations. See below a full demonstration of the method:

In [25]:
# USING SYMBOLIC COMPUTATION TO 
# FIND GOOD INPUT MATRICES SUCH THAT THE QR DECOMPOSITION
# THROUGH HOUSEHOLDER TRANSFORMATIONS RENDER RATIONAL COMPUTATIONS
#
# Ana C. C. Couto, David J. Jeffrey
# Dept. of Applied Math. Western University

import numpy as np
import sympy as sym
from fractions import Fraction
from scipy import linalg

#------------------------------------------------------------------------------
# CALCULATING THE HOUSEHOLDER QR DECOMPOSITION OF A
#------------------------------------------------------------------------------ 

# Input elements of A's 1st column
# a11,a21,a31,a41 must form a pythagorean quadruple
a11,a21,a31,a41 = sym.symbols("a11 a21 a31 a41")

# Define elements of A's 2nd and 3rd columns, that can assume many values 
# to satisfy the condition of rendering rational computations
a12,a22,a32,a42 = sym.symbols("a12 a22 a32 a42")
a13,a23,a33,a43 = sym.symbols("a13 a23 a33 a43")

# Input elements of A's 4th column (can be any set of rationals)
a14,a24,a34,a44 = sym.symbols("a14 a24 a34 a44")

# Build A matrix from components above
A = sym.Matrix([[a11,a12,a13,a14],[a21,a22,a23,a24],[a31,a32,a33,a34],\
    [a41,a42,a43,a44]])

# FIRST HOUSEHOLDER TRANSFORMATION
# Define the 1st Householder target vector z1 and displacement vector v1
z1  = sym.Matrix([a11,a21,a31,a41])
v1  = sym.sqrt((a11**2+a21**2+a31**2+a41**2))
v1  = sym.Matrix([v1,0,0,0]) - z1

# Calculate the 1st Householder Transformation H1*A
vvT1 = v1*v1.T
vTv1 = v1.T.dot(v1)
H1   = sym.eye(4) - (2/vTv1)*vvT1
H1A  = H1*A

# SECOND HOUSEHOLDER TRANSFORMATION
# Define the 2nd Householder target vector z2 and displacement vector v2
z2  = sym.Matrix([H1A[1,1],H1A[2,1],H1A[3,1]])
v2  = sym.sqrt((H1A[1,1]**2+H1A[2,1]**2+H1A[3,1]**2))
v2  = sym.Matrix([v2,0,0]) - z2

# Calculate the 2nd Householder Transformation H2*H1*A
vvT2 = v2*v2.T
vTv2 = v2.T.dot(v2)
H2   = sym.eye(3) - (2/vTv2)*vvT2
H2H1A = H2*H1A[1:,1:]
Q2Q1A = sym.Matrix([[H1A[0,0],H1A[0,1],H1A[0,2],H1A[0,3]], \
        [H1A[1,0],H2H1A[0,0],H2H1A[0,1],H2H1A[0,2]],\
        [H1A[2,0],H2H1A[1,0],H2H1A[1,1],H2H1A[1,2]], \
        [H1A[3,0],H2H1A[2,0],H2H1A[2,1],H2H1A[2,2]]])

# THIRD HOUSEHOLDER TRANSFORMATION
# Define the 3rd Householder target vector z3 and displacement vector v3
z3  = sym.Matrix([H2H1A[1,1],H2H1A[2,1]])
v3  = sym.sqrt((H2H1A[1,1]**2+H2H1A[2,1]**2))
v3  = sym.Matrix([v3,0]) - z3

# Calculate the 3rd Householder Transformation H3*H2*H1*A
vvT3 = v3*v3.T
vTv3 = v3.T.dot(v3)
H3   = sym.eye(2) - (2/vTv3)*vvT3
H3H2H1A = H3*H2H1A[1:,1:]
list1 = [Q2Q1A[2,0],Q2Q1A[2,1],H3H2H1A[0,0],H3H2H1A[0,1]]
Q3Q2Q1A = sym.Matrix([[Q2Q1A[0,0],Q2Q1A[0,1],Q2Q1A[0,2],Q2Q1A[0,3]],\
          [Q2Q1A[1,0],Q2Q1A[1,1],Q2Q1A[1,2],Q2Q1A[1,3]], \
          [Q2Q1A[2,0],Q2Q1A[2,1],\
           H3H2H1A[0,0],H3H2H1A[0,1]],\
          [Q2Q1A[3,0],Q2Q1A[3,1],H3H2H1A[1,0],H3H2H1A[1,1]]])

#-------------------------------------------------------------------------------
# SEARCHING FOR SUITABLE 2ND AND 3RD COLUMNS
#-------------------------------------------------------------------------------

# Define the 1st and 2nd target vector norms
target_norm1 = sym.sqrt(H1A[1,1]**2 + H1A[2,1]**2 + H1A[3,1]**2)
target_norm2 = sym.sqrt(H2H1A[1,1]**2 + H2H1A[2,1]**2)

# Creating lists of rational numbers of the shape a/b in which a, b  
# are integers from 0,1 up to a pre-defined value
a = np.linspace(0,5,6)
b = np.linspace(1,5,5)

Q_list = []
for m in b:
    for n in a:
        if sym.Rational(n,m) not in Q_list:
            Q_list.append(sym.Rational(n,m))

# Defining a function to search the Q_list for rational 2nd columns such
# that |z1| (target_norm1) is an integer and |v1| is not 0 (so that there is no
# division by 0 when calculating H1). The inputs are A's 1st and 4th columns
def suitable_2col(A11,A21,A31,A41):
 for k in Q_list:
  for j in Q_list:
   for i in Q_list:
    for l in Q_list:
     if int(target_norm1.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),\
      (a12,k),(a22,j),(a32,i),(a42,l)])) ==\
      target_norm1.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),\
      (a12,k),(a22,j),(a32,i),(a42,l)]):
      if H1A[3,1].subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),\
       (a12,k),(a22,j),(a32,i),(a42,l)]) != 0:
         print(k,j,i,l)
  return

# Defining a function to search the Q_list for rational 3rd columns such
# that |z2| (target_norm2) is an integer and |v2| is not 0 (so that there is no
# division by 0 when calculating H2). The inputs are A's 1st and 4th columns
def suitable_3col(A11,A21,A31,A41, A12,A22,A32,A42):
 for k in Q_list:
  for j in Q_list:
   for i in Q_list:
    for l in Q_list:
     if int(target_norm2.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),\
      (a12,A12),(a22,A22),(a32,A32),(a42,A42),\
      (a13,k),(a23,j),(a33,i),(a43,l)]))==\
      target_norm2.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),\
      (a12,A12),(a22,A22),(a32,A32),(a42,A42),\
      (a13,k),(a23,j),(a33,i),(a43,l)]):
      if H2H1A[2,1].subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),\
       (a12,A12),(a22,A22),(a32,A32),(a42,A42),\
       (a13,k),(a23,j),(a33,i),(a43,l)]) != 0:
         print(k,j,i,l)
  return
#------------------------------------------------------------------------------
# RUNNING THE TESTS AND PRINTING THE RESULTS
#------------------------------------------------------------------------------

# Calculate Q and R matrices from the Householder Transformations and check:
# 0. If A is invertible
# 1. If the intermediate Householder Transformations will work as expected
# 2. If the components of Q and R will be rational
# 3. If R will in fact be upper triangular
# 4. If Q will in fact be orthonormal 

R = Q3Q2Q1A 
Q = (R*(A**-1)).T

def test(A11,A21,A31,A41, A12,A22,A32,A42, A13,A23,A33,A43, A14,A24,A34,A44):
# Check if A is invertible
  Asubs = A.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),(a12,A12),\
        (a22,A22),(a32,A32),(a42,A42),(a13,A13),(a23,A23),(a33,A33),\
        (a43,A43),(a14,A14),(a24,A24),(a34,A34),(a44,A44)])

  if Asubs.det() == 0:
    print('A is not invertible. Please, redefine the 4th column')

  if Asubs.det() != 0:
# Check if the intermediate Householder Transformations will work as expected
    print('\n This is the first updated Matrix Q1*A:\n')
    print(H1A.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),(a12,A12),\
        (a22,A22),(a32,A32),(a42,A42),(a13,A13),(a23,A23),(a33,A33),\
        (a43,A43),(a14,A14),(a24,A24),(a34,A34),(a44,A44)]))

    print('\n This is the second updated Matrix Q2*Q1*A:\n')
    print(Q2Q1A.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),(a12,A12),\
        (a22,A22),(a32,A32),(a42,A42),(a13,A13),(a23,A23),(a33,A33),\
        (a43,A43),(a14,A14),(a24,A24),(a34,A34),(a44,A44)]))

# Check if the components of Q and R will be rational
    print('\n This is the upper triangular R matrix:\n')
    Rsubs = R.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),(a12,A12),\
        (a22,A22),(a32,A32),(a42,A42),(a13,A13),(a23,A23),(a33,A33),\
        (a43,A43),(a14,A14),(a24,A24),(a34,A34),(a44,A44)])
    sym.pprint(Rsubs)

    print('\n This is the orthonormal Q matrix:\n')
    Qsubs = Q.subs([(a11,A11),(a21,A21),(a31,A31),(a41,A41),(a12,A12),\
        (a22,A22),(a32,A32),(a42,A42),(a13,A13),(a23,A23),(a33,A33),\
        (a43,A43),(a14,A14),(a24,A24),(a34,A34),(a44,A44)])
    sym.pprint(Qsubs)

# Check if Q will in fact be orthonormal 
    print('\n The dot product between the columns of Q are equal to:\n')
    dot1 = Qsubs.col(0).dot(Qsubs.col(1))
    dot2 = Qsubs.col(0).dot(Qsubs.col(2))
    dot3 = Qsubs.col(0).dot(Qsubs.col(3))
    dot4 = Qsubs.col(1).dot(Qsubs.col(2))
    dot5 = Qsubs.col(1).dot(Qsubs.col(3))
    dot6 = Qsubs.col(2).dot(Qsubs.col(3))
    print(dot1,dot2,dot3,dot4,dot5,dot6)

    print('\n The norm of the columns of Q are equal to:\n')
    norm1 = sym.sqrt(Qsubs[0,0]**2+Qsubs[1,0]**2+Qsubs[2,0]**2+Qsubs[3,0]**2)
    norm2 = sym.sqrt(Qsubs[0,1]**2+Qsubs[1,1]**2+Qsubs[2,1]**2+Qsubs[3,1]**2)
    norm3 = sym.sqrt(Qsubs[0,2]**2+Qsubs[1,2]**2+Qsubs[2,2]**2+Qsubs[3,2]**2)
    norm4 = sym.sqrt(Qsubs[0,3]**2+Qsubs[1,3]**2+Qsubs[2,3]**2+Qsubs[3,3]**2)
    print(norm1,norm2,norm3,norm4)

# Lastly, last check that QR in fact results back to A
    print('\n is Q*R = A? \n')
    question = Qsubs*Rsubs == Asubs
    print(question)
    if (question==True):
        print('\n This is the matrix you will want to use as input:\n')
        sym.pprint(sym.Matrix([[A11,A21,A31,A41], [A12,A22,A32,A42], [A13,A23,A33,A43], [A14,A24,A34,A44]]).T)

    return

#### Now, I'm going to demonstrate how to use the program above to find an input matrix for the Householder method.

The first column has to be a pythagorean tuple[1]. This just means that the norm of the elements has to evaluate to a rational number. Let us gather a list of suitable second columns for a 4x4 matrix whose first column is (3,0,4,12).

In [19]:
suitable_2col(3,0,4,12)

0 0 1 4
0 0 4 3
0 0 4/3 1
0 4 4 3
0 4/5 4/5 3/5


Ok, we can see that the suitable_2col function spit out 5 possible outcomes for us under the constraints that we provided for the search. Now, let us pick one out of those 5, say, (0,4,4,3), and find a suitable third column for our matrix: 

In [6]:
suitable_3col(3,0,4,12,0,4,4,3)

0 0 1 4
0 0 5/3 5/4
0 2 1/3 1/4
0 3 4/3 1
0 4 2/3 1/2
0 5 0 0
0 5 5/3 5/4
0 1/3 2 3/2
0 2/3 4 3
0 5/3 0 0


This time, we have 10 possible formations for a column. If we pick one, say, (0 3 4/3 1), we have built a full 4x4 matrix, since the fourth column could be any combination of rational numbers[1]. Now, let us apply the test function to the colums we've chosen so far plus a random fourth column (1,1,1,1) and check if the generated matrix in fact renders rational Householder calculations: 

In [26]:
test(3,0,4,12,0,4,4,3,0,3,Fraction(4,3),1,1,1,1,1)


 This is the first updated Matrix Q1*A:

Matrix([[13, 4, 4/3, 19/13], [0, 4, 3, 1], [0, 12/5, 4/5, 53/65], [0, -9/5, -3/5, 29/65]])

 This is the second updated Matrix Q2*Q1*A:

Matrix([[13, 4, 4/3, 19/13], [0, 5, 3, 67/65], [0, 0, 4/5, 241/325], [0, 0, -3/5, 163/325]])

 This is the upper triangular R matrix:

⎡             19 ⎤
⎢13  4  4/3   ── ⎥
⎢             13 ⎥
⎢                ⎥
⎢             67 ⎥
⎢0   5   3    ── ⎥
⎢             65 ⎥
⎢                ⎥
⎢             19 ⎥
⎢0   0   1    ── ⎥
⎢             65 ⎥
⎢                ⎥
⎢            -11 ⎥
⎢0   0   0   ────⎥
⎣             13 ⎦

 This is the orthonormal Q matrix:

⎡      -12     16   -12  ⎤
⎢3/13  ────    ──   ──── ⎥
⎢       65     65    13  ⎥
⎢                        ⎥
⎢ 0     4/5   3/5     0  ⎥
⎢                        ⎥
⎢       36    -48        ⎥
⎢4/13   ──    ────  -3/13⎥
⎢       65     65        ⎥
⎢                        ⎥
⎢ 12           12        ⎥
⎢ ──   -9/65   ──   4/13 ⎥
⎣ 13           65        ⎦

 The dot pro

#### The matrix above will produce rational (and easy to carry out) calculations. And you can use it when designing excercises. The tests also check if the decomposition worked correctly. 

You are welcome to tweak the functions' inputs and rerun it as you wish in order to build another input matrix, as well as change the limits from the list search of the original code if you want to expand the range of numbers to look for in the columns. If you wish to know more about the concepts around rational QR factorization, I would advise you to reed the referenced paper. 

There are tons of material on Householder and other QR decomposition methods available on mathematical and scientifical archives if that is what you are interested in. For any questions on the algorithm presented, you can contact me on anacamargos11@gmail.com